In [1]:
# imports
# initialization - importing the required library
import pandas as pd 
import numpy as np
import requests
import json
import os

# a hack to print all my cell outputs / change 'all' to 'last_expr' to revert to default
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# importing the saved stations_df 
stations_df = pd.read_csv('../data/stations_df.csv')
stations_df.head()

,Station Name,Latitude,Longitude,Number of Bikes,Station ID
0,Orient Heights T Stop - Bennington St at Sarat...,42.386781,-71.006098,19,553ed0300d38108b4f21a6bafa3db70c
1,Maverick Square - Lewis Mall,42.368844,-71.039778,33,93542dcbf21f5411569adb92cd7cc199
2,East Boston Neighborhood Health Center - 20 Ma...,42.369536,-71.039431,16,d9c7ef5dbda4ed944d1bf51fe540acb6
3,Bennington St at Byron St,42.383533,-71.016191,15,0568389e659e679fbe29a5ac12cd49c0
4,Boston East - 126 Border St,42.373312,-71.041020,15,47b79abc28a54d0e4689b1096ceb8466


# Foursquare

## Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Resource: https://location.foursquare.com/places/docs/categories for FourSquare category listing.

        - Restaurant - 13065
        - Car rentals (my selected POI) - 19048

In [4]:

# function 
def foursquares_api(ll, searchitem): # latitude, longitude
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "categories":searchitem, 
        "ll": ll,
        "sort":"DISTANCE",
        "radius":"1000",
        "limit":"50" 
    }

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }

    response = requests.request("GET", url, params=params, headers=headers)
    
    return response.json()

# relevant categories:
# restaurant: 13065
# car rental : 19048

## Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
# Analyze one result first
idx = 100
ll = stations_df.loc[idx, 'Latitude'].astype(str) + ',' + stations_df.loc[idx, 'Longitude'].astype(str)
response = foursquares_api(ll, 13065)
response

{'results': [{'fsq_id': '613c28207bce485472c78e47',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 155,
   'geocodes': {'main': {'latitude': 42.347313, 'longitude': -71.043566}},
   'link': '/v3/places/613c28207bce485472c78e47',
   'location': {'address': '450 Summer St',
    'census_block': '250250606031021',
    'country': 'US',
    'cross_street': '',
    'dma': 'Boston',
    'formatted_address': '450 Summer St, Boston, MA 02210',
    'locality': 'Boston',
    'postcode': '02210',
    'region': 'MA'},
   'name': 'Coquette',
   'related_places': {'parent': {'fsq_id': '5c0710dd2d2fd9002c334c1a',
     'name': 'Omni Hotel at the Seaport Parkin'}},
   'timezone': 'America/New_York'},
  {'fsq_id': '528feaae11d20de9fd0955bd',
   'categories': [{'id': 13068,
     'name': 'American Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img

In [24]:
# user-defined function to analyze json files
def analyze_json (response_json):
    # Get the name of 'response_json'
    name_of_response_json = ''
    for name, value in globals().items():
        if value is response_json:
            name_of_response_json = name


    print (f"Analyzing {name_of_response_json}\n========================\n")
    if type(response_json) == dict:
        print("Type: Dictionary\nKey List:\n")
        for key in response_json.keys():
            print(f"Key name: {key}\nKey type: {type(response_json[key])}")
    elif type(response_json) == list:
        print(f"Type: List\n\Size: {len(response_json)}")
    print("\n\n\n")

In [7]:
analyze_json (response) # dictionary of 2 keys including results
analyze_json (response['results']) # list of length 5
analyze_json (response['results'][0])

Analyzing response

Type: Dictionary
Key List:

Key name: results
Key type: <class 'list'>
Key name: context
Key type: <class 'dict'>




Analyzing 

Type: List
\Size: 5




Analyzing 

Type: Dictionary
Key List:

Key name: fsq_id
Key type: <class 'str'>
Key name: categories
Key type: <class 'list'>
Key name: chains
Key type: <class 'list'>
Key name: distance
Key type: <class 'int'>
Key name: geocodes
Key type: <class 'dict'>
Key name: link
Key type: <class 'str'>
Key name: location
Key type: <class 'dict'>
Key name: name
Key type: <class 'str'>
Key name: related_places
Key type: <class 'dict'>
Key name: timezone
Key type: <class 'str'>






In [8]:
result1 = analyze_json (response['results'][3]) # random result
analyze_json(result1)

Analyzing 

Type: Dictionary
Key List:

Key name: fsq_id
Key type: <class 'str'>
Key name: categories
Key type: <class 'list'>
Key name: chains
Key type: <class 'list'>
Key name: distance
Key type: <class 'int'>
Key name: geocodes
Key type: <class 'dict'>
Key name: link
Key type: <class 'str'>
Key name: location
Key type: <class 'dict'>
Key name: name
Key type: <class 'str'>
Key name: related_places
Key type: <class 'dict'>
Key name: timezone
Key type: <class 'str'>




Analyzing result1







From the analysis, the relevant data appear to be: `name` and `categories`

Put your parsed results into a DataFrame

# Put your parsed results into a DataFrame

### Car rentals (POI)

In [5]:
# to fill up a dictionary, which will be converted to a pandas DataFrame
bike_id = []
names = []
categories = []
counter = 0
lenght_of_df = len(stations_df)


for idx in list(stations_df.index):
# for idx in [100]: # line from testing
    ll = stations_df.loc[idx, 'Latitude'].astype(str) + ',' + stations_df.loc[idx, 'Longitude'].astype(str)
    print (f'Processing row {idx}.......... \t {round((idx*100)/lenght_of_df)}% complete') # for monitoring
    response = foursquares_api(ll, 19048)
    
    
    for result in response['results']:
        bike_id.append(idx)
        names.append(result['name'])
        categories.append(result['categories'])

Processing row 0.......... 	 0% complete
Processing row 1.......... 	 0% complete
Processing row 2.......... 	 0% complete
Processing row 3.......... 	 1% complete
Processing row 4.......... 	 1% complete
Processing row 5.......... 	 1% complete
Processing row 6.......... 	 1% complete
Processing row 7.......... 	 2% complete
Processing row 8.......... 	 2% complete
Processing row 9.......... 	 2% complete
Processing row 10.......... 	 2% complete
Processing row 11.......... 	 2% complete
Processing row 12.......... 	 3% complete
Processing row 13.......... 	 3% complete
Processing row 14.......... 	 3% complete
Processing row 15.......... 	 3% complete
Processing row 16.......... 	 4% complete
Processing row 17.......... 	 4% complete
Processing row 18.......... 	 4% complete
Processing row 19.......... 	 4% complete
Processing row 20.......... 	 5% complete
Processing row 21.......... 	 5% complete
Processing row 22.......... 	 5% complete
Processing row 23.......... 	 5% complete
Pr

Processing row 190.......... 	 43% complete
Processing row 191.......... 	 43% complete
Processing row 192.......... 	 43% complete
Processing row 193.......... 	 44% complete
Processing row 194.......... 	 44% complete
Processing row 195.......... 	 44% complete
Processing row 196.......... 	 44% complete
Processing row 197.......... 	 44% complete
Processing row 198.......... 	 45% complete
Processing row 199.......... 	 45% complete
Processing row 200.......... 	 45% complete
Processing row 201.......... 	 45% complete
Processing row 202.......... 	 46% complete
Processing row 203.......... 	 46% complete
Processing row 204.......... 	 46% complete
Processing row 205.......... 	 46% complete
Processing row 206.......... 	 47% complete
Processing row 207.......... 	 47% complete
Processing row 208.......... 	 47% complete
Processing row 209.......... 	 47% complete
Processing row 210.......... 	 47% complete
Processing row 211.......... 	 48% complete
Processing row 212.......... 	 4

Processing row 377.......... 	 85% complete
Processing row 378.......... 	 85% complete
Processing row 379.......... 	 86% complete
Processing row 380.......... 	 86% complete
Processing row 381.......... 	 86% complete
Processing row 382.......... 	 86% complete
Processing row 383.......... 	 86% complete
Processing row 384.......... 	 87% complete
Processing row 385.......... 	 87% complete
Processing row 386.......... 	 87% complete
Processing row 387.......... 	 87% complete
Processing row 388.......... 	 88% complete
Processing row 389.......... 	 88% complete
Processing row 390.......... 	 88% complete
Processing row 391.......... 	 88% complete
Processing row 392.......... 	 88% complete
Processing row 393.......... 	 89% complete
Processing row 394.......... 	 89% complete
Processing row 395.......... 	 89% complete
Processing row 396.......... 	 89% complete
Processing row 397.......... 	 90% complete
Processing row 398.......... 	 90% complete
Processing row 399.......... 	 9

In [6]:
# creating DataFrame
foursquares_rentals_df = pd.DataFrame({
    'Name': names,
    'Categories': categories,
    'Bike Station Id': bike_id
    
})
# converting the values in the Categories column into a list, not a dictionary
cats = []
for row in list(foursquares_rentals_df.index):
    cat = [category['name'] for category in foursquares_rentals_df['Categories'][row]]
    cats.append(cat)
    
foursquares_rentals_df['Category'] = cats
foursquares_rentals_df.drop(['Categories'], axis=1, inplace=True)

#### cleaning the dataframe

In [11]:
foursquares_rentals_df.head()
len(foursquares_rentals_df)

,Name,Bike Station Id,Category
0,Avis Rent A Car,0,[Rental Car Location]
1,Avis Rent A Car,0,[Rental Car Location]
2,Budget Truck Rental,0,[Rental Car Location]
3,Advantage Rent A Car,0,[Rental Car Location]
4,Enterprise Rent-A-Car,0,[Rental Car Location]


3961

In [12]:
# checking for null data - No null data
foursquares_rentals_df.info() 
# checking for duplicates
len(foursquares_rentals_df) - len(foursquares_rentals_df.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=True))
# - 922

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3961 entries, 0 to 3960
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             3961 non-null   object
 1   Bike Station Id  3961 non-null   int64 
 2   Category         3961 non-null   object
dtypes: int64(1), object(2)
memory usage: 93.0+ KB


922

In [13]:
# Deleting duplicates
foursquares_rentals_df.drop_duplicates(subset=['Name', 'Bike Station Id'], inplace=True, ignore_index=True)

In [15]:
# saving the file
foursquares_rentals_df.to_csv('../data/foursquares_car_rentals_df.csv', index=False)

pd.read_csv('../data/foursquares_car_rentals_df.csv')

,Name,Bike Station Id,Category
0,Avis Rent A Car,0,['Rental Car Location']
1,Budget Truck Rental,0,['Rental Car Location']
2,Advantage Rent A Car,0,['Rental Car Location']
3,Enterprise Rent-A-Car,0,['Rental Car Location']
4,Flight Car,0,['Rental Car Location']
...,...,...,...
3034,Zipcar Park Sq/Motor Mart Garage,442,['Rental Car Location']
3035,Dollar Car Rental,442,['Rental Car Location']
3036,Zipcar Huntington Ave/Cumberland,442,['Rental Car Location']
3037,Thrifty Car Rental,442,['Rental Car Location']


### Restaurants

In [ ]:
# running the same for loop as above, but category now set to 13065

In [16]:
# creating a for loop to fill up a dictionary, which will be converted to a pandas DataFrame
bike_id = []
names = []
categories = []
counter = 0
lenght_of_df = len(stations_df)


for idx in list(stations_df.index):
# for idx in [100]: # line from testing
    ll = stations_df.loc[idx, 'Latitude'].astype(str) + ',' + stations_df.loc[idx, 'Longitude'].astype(str)
    print (f'Processing row {idx}.......... \t {round((idx*100)/lenght_of_df)}% complete') # for monitoring
    response = foursquares_api(ll, 13065)
    
    
    for result in response['results']:
        bike_id.append(idx)
        names.append(result['name'])
        categories.append(result['categories'])

Processing row 0.......... 	 0% complete
Processing row 1.......... 	 0% complete
Processing row 2.......... 	 0% complete
Processing row 3.......... 	 1% complete
Processing row 4.......... 	 1% complete
Processing row 5.......... 	 1% complete
Processing row 6.......... 	 1% complete
Processing row 7.......... 	 2% complete
Processing row 8.......... 	 2% complete
Processing row 9.......... 	 2% complete
Processing row 10.......... 	 2% complete
Processing row 11.......... 	 2% complete
Processing row 12.......... 	 3% complete
Processing row 13.......... 	 3% complete
Processing row 14.......... 	 3% complete
Processing row 15.......... 	 3% complete
Processing row 16.......... 	 4% complete
Processing row 17.......... 	 4% complete
Processing row 18.......... 	 4% complete
Processing row 19.......... 	 4% complete
Processing row 20.......... 	 5% complete
Processing row 21.......... 	 5% complete
Processing row 22.......... 	 5% complete
Processing row 23.......... 	 5% complete
Pr

Processing row 190.......... 	 43% complete
Processing row 191.......... 	 43% complete
Processing row 192.......... 	 43% complete
Processing row 193.......... 	 44% complete
Processing row 194.......... 	 44% complete
Processing row 195.......... 	 44% complete
Processing row 196.......... 	 44% complete
Processing row 197.......... 	 44% complete
Processing row 198.......... 	 45% complete
Processing row 199.......... 	 45% complete
Processing row 200.......... 	 45% complete
Processing row 201.......... 	 45% complete
Processing row 202.......... 	 46% complete
Processing row 203.......... 	 46% complete
Processing row 204.......... 	 46% complete
Processing row 205.......... 	 46% complete
Processing row 206.......... 	 47% complete
Processing row 207.......... 	 47% complete
Processing row 208.......... 	 47% complete
Processing row 209.......... 	 47% complete
Processing row 210.......... 	 47% complete
Processing row 211.......... 	 48% complete
Processing row 212.......... 	 4

Processing row 377.......... 	 85% complete
Processing row 378.......... 	 85% complete
Processing row 379.......... 	 86% complete
Processing row 380.......... 	 86% complete
Processing row 381.......... 	 86% complete
Processing row 382.......... 	 86% complete
Processing row 383.......... 	 86% complete
Processing row 384.......... 	 87% complete
Processing row 385.......... 	 87% complete
Processing row 386.......... 	 87% complete
Processing row 387.......... 	 87% complete
Processing row 388.......... 	 88% complete
Processing row 389.......... 	 88% complete
Processing row 390.......... 	 88% complete
Processing row 391.......... 	 88% complete
Processing row 392.......... 	 88% complete
Processing row 393.......... 	 89% complete
Processing row 394.......... 	 89% complete
Processing row 395.......... 	 89% complete
Processing row 396.......... 	 89% complete
Processing row 397.......... 	 90% complete
Processing row 398.......... 	 90% complete
Processing row 399.......... 	 9

In [17]:
# as for car rentals above, creating the DataFrame and formatting the categories section.
# creating DataFrame
foursquares_restaurant_df = pd.DataFrame({
    'Name': names,
    'Categories': categories,
    'Bike Station Id': bike_id
    
})
# converting the values in the Categories column into a list, not a dictionary
cats = []

for row in list(foursquares_restaurant_df.index):
    cat = [category['name'] for category in foursquares_restaurant_df['Categories'][row]]
    cats.append(cat)
    
foursquares_restaurant_df['Category'] = cats
foursquares_restaurant_df.drop(['Categories'], axis=1, inplace=True)

In [20]:
foursquares_restaurant_df

,Name,Bike Station Id,Category
0,El Paisa Orient Heights,0,"[Latin American Restaurant, Mexican Restaurant]"
1,Angela's Cafe Orient Heights,0,[Mexican Restaurant]
2,El Paisa Butchery,0,"[Deli, Meat and Seafood Store]"
3,Renegade's Pub,0,"[Cocktail Bar, Pub, Burger Joint]"
4,E. Umi Sushi,0,[Sushi Restaurant]
...,...,...,...
19285,Cortes Street,442,[Restaurant]
19286,Nusr-et Steakhouse,442,[Steakhouse]
19287,Trophy Room,442,"[Cocktail Bar, American Restaurant]"
19288,Thornton's Restaurant,442,"[Café, Diner, American Restaurant]"


#### cleaning the dataframe

In [18]:
# checking for null data - No null data
foursquares_restaurant_df.info() 
# checking for duplicates
len(foursquares_restaurant_df) - len(foursquares_restaurant_df.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=True))
# - 295 duplicates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19290 entries, 0 to 19289
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             19290 non-null  object
 1   Bike Station Id  19290 non-null  int64 
 2   Category         19290 non-null  object
dtypes: int64(1), object(2)
memory usage: 452.2+ KB


295

In [22]:
# Deleting duplicates
foursquares_restaurant_df.drop_duplicates(subset=['Name', 'Bike Station Id'], inplace=True, ignore_index=True)
foursquares_restaurant_df

,Name,Bike Station Id,Category
0,El Paisa Orient Heights,0,"[Latin American Restaurant, Mexican Restaurant]"
1,Angela's Cafe Orient Heights,0,[Mexican Restaurant]
2,El Paisa Butchery,0,"[Deli, Meat and Seafood Store]"
3,Renegade's Pub,0,"[Cocktail Bar, Pub, Burger Joint]"
4,E. Umi Sushi,0,[Sushi Restaurant]
...,...,...,...
18990,Cortes Street,442,[Restaurant]
18991,Nusr-et Steakhouse,442,[Steakhouse]
18992,Trophy Room,442,"[Cocktail Bar, American Restaurant]"
18993,Thornton's Restaurant,442,"[Café, Diner, American Restaurant]"


In [24]:
# saving the file
foursquares_restaurant_df.to_csv('../data/foursquares_restaurant_df.csv', index=False) 

pd.read_csv('../data/foursquares_restaurant_df.csv')

,Name,Bike Station Id,Category
0,El Paisa Orient Heights,0,"['Latin American Restaurant', 'Mexican Restaur..."
1,Angela's Cafe Orient Heights,0,['Mexican Restaurant']
2,El Paisa Butchery,0,"['Deli', 'Meat and Seafood Store']"
3,Renegade's Pub,0,"['Cocktail Bar', 'Pub', 'Burger Joint']"
4,E. Umi Sushi,0,['Sushi Restaurant']
...,...,...,...
18990,Cortes Street,442,['Restaurant']
18991,Nusr-et Steakhouse,442,['Steakhouse']
18992,Trophy Room,442,"['Cocktail Bar', 'American Restaurant']"
18993,Thornton's Restaurant,442,"['Café', 'Diner', 'American Restaurant']"


# Yelp

**Resources** :
    
- https://docs.developer.yelp.com/reference/v3_business_search
- https://docs.developer.yelp.com/docs/resources-categories

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

For car rentals: carrentals
    
For restaurants: restaurant

In [18]:
def yelp_api(Latitude, Longitude, searchterm): 
    print ('yelpapi:', searchterm)
    """
    Function to call the Yelp API:
        inputs latitude and longitude of interest (float numbers)
        and search term as string
        returns reponse.json()
    """

    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        
        "latitude": Latitude,
        "longitude": Longitude,
        "term":searchterm,
        "radius":"1000",
        "sort_by":"best_match",
        "limit":"20"
    }
    headers = {
        "Authorization": ("Bearer " + str(YELP_API_KEY)), 
        "Accept": "application/json",
    }

    response = requests.request("GET", url, params=params, headers=headers)
    return response.json()

# searchterms: carrental, restaurant

## Parse through the response
to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [39]:
idx = 250 # random station
Latitude = stations_df.loc[idx, 'Latitude']
Longitude = stations_df.loc[idx, 'Longitude']

yelp_response1 = yelp_api(Latitude, Longitude, 'carrentals')

yelpapi: carrentals


In [40]:
yelp_response1

{'businesses': [{'id': 'L2u8JaShvZZtrYLCdMsPkw',
   'alias': 'thrifty-car-rental-boston-hotel',
   'name': 'Thrifty Car Rental',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/HbJnsW_ud3rAx9DsZmqkng/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/thrifty-car-rental-boston-hotel?adjust_creative=6_pP7_hHjyiYNWKp8rEecw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=6_pP7_hHjyiYNWKp8rEecw',
   'review_count': 39,
   'categories': [{'alias': 'carrental', 'title': 'Car Rental'}],
   'rating': 1.5,
   'coordinates': {'latitude': 42.372518, 'longitude': -71.122122},
   'transactions': [],
   'location': {'address1': '110 Mt Auburn St',
    'address2': '',
    'address3': 'Harvard Square Hotel',
    'city': 'BOSTON, HOTEL',
    'zip_code': '02138',
    'country': 'US',
    'state': 'MA',
    'display_address': ['110 Mt Auburn St',
     'Harvard Square Hotel',
     'BOSTON, HOTEL, MA 02138']},
   'phone': '+16178762758',
   'display_phone': '(6

In [41]:
type(yelp_response1)

dict

In [42]:
# parsing through file to find the relevant data

analyze_json(yelp_response1)

Analyzing _40

Type: Dictionary
Key List:

Key name: businesses
Key type: <class 'list'>
Key name: total
Key type: <class 'int'>
Key name: region
Key type: <class 'dict'>






In [43]:
analyze_json(yelp_response1['businesses'])

analyze_json(yelp_response1['businesses'][0])

yelp_response1['businesses'][0].keys()

yelp_response1['businesses'][0]['categories']

yelp_response1['businesses'][0]['rating']

yelp_response1['businesses'][0]['review_count']

yelp_response1['businesses'][0]['name']

yelp_response1['businesses'][0]['id']

yelp_response1['businesses'][0]['alias']

yelp_response1['businesses'][0]['rating']

Analyzing 

Type: List
\Size: 1




Analyzing 

Type: Dictionary
Key List:

Key name: id
Key type: <class 'str'>
Key name: alias
Key type: <class 'str'>
Key name: name
Key type: <class 'str'>
Key name: image_url
Key type: <class 'str'>
Key name: is_closed
Key type: <class 'bool'>
Key name: url
Key type: <class 'str'>
Key name: review_count
Key type: <class 'int'>
Key name: categories
Key type: <class 'list'>
Key name: rating
Key type: <class 'float'>
Key name: coordinates
Key type: <class 'dict'>
Key name: transactions
Key type: <class 'list'>
Key name: location
Key type: <class 'dict'>
Key name: phone
Key type: <class 'str'>
Key name: display_phone
Key type: <class 'str'>
Key name: distance
Key type: <class 'float'>






dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'location', 'phone', 'display_phone', 'distance'])

[{'alias': 'carrental', 'title': 'Car Rental'}]

1.5

39

'Thrifty Car Rental'

'L2u8JaShvZZtrYLCdMsPkw'

'thrifty-car-rental-boston-hotel'

1.5

Based on parsing, the relevant information imo is: `name`, `id`, `ratings`, `categories`

## Putting parsed results into a DataFrame

### POI: Car Rentals

In [44]:
# running for the following search terms carrentals
def loop_yelp_api_by_search (query):

    bike_id = []
    names = []
    reviews = []
    ratings = []
    categories = []
    lenght_of_df = len(stations_df)

    for idx in list(stations_df.index):
        lat = stations_df.loc[idx, 'Latitude']
        long = stations_df.loc[idx, 'Longitude']
        print (f'Processing row {idx}.......... \t {round((idx*100)/lenght_of_df)}% complete')
        response = yelp_api(lat, long, query)

        for business in response['businesses']:
            bike_id.append(idx)
            names.append(business['name'])
            reviews.append(business['review_count'])
            ratings.append(business['rating'])
            categories.append(business['categories'])

    # creating the dictionary, then the DataFrame
    df = pd.DataFrame({
        'Name': names,
        'Review Count': reviews,
        'Ratings': ratings,
        'Categories': categories,
        'Bike Station Id': bike_id
     }) 
    return df   


In [45]:
yelp_rentals = loop_yelp_api_by_search('carrentals')
yelp_rentals

Processing row 0.......... 	 0% complete
yelpapi: carrentals
Processing row 1.......... 	 0% complete
yelpapi: carrentals
Processing row 2.......... 	 0% complete
yelpapi: carrentals
Processing row 3.......... 	 1% complete
yelpapi: carrentals
Processing row 4.......... 	 1% complete
yelpapi: carrentals
Processing row 5.......... 	 1% complete
yelpapi: carrentals
Processing row 6.......... 	 1% complete
yelpapi: carrentals
Processing row 7.......... 	 2% complete
yelpapi: carrentals
Processing row 8.......... 	 2% complete
yelpapi: carrentals
Processing row 9.......... 	 2% complete
yelpapi: carrentals
Processing row 10.......... 	 2% complete
yelpapi: carrentals
Processing row 11.......... 	 2% complete
yelpapi: carrentals
Processing row 12.......... 	 3% complete
yelpapi: carrentals
Processing row 13.......... 	 3% complete
yelpapi: carrentals
Processing row 14.......... 	 3% complete
yelpapi: carrentals
Processing row 15.......... 	 3% complete
yelpapi: carrentals
Processing row 16.

Processing row 131.......... 	 30% complete
yelpapi: carrentals
Processing row 132.......... 	 30% complete
yelpapi: carrentals
Processing row 133.......... 	 30% complete
yelpapi: carrentals
Processing row 134.......... 	 30% complete
yelpapi: carrentals
Processing row 135.......... 	 30% complete
yelpapi: carrentals
Processing row 136.......... 	 31% complete
yelpapi: carrentals
Processing row 137.......... 	 31% complete
yelpapi: carrentals
Processing row 138.......... 	 31% complete
yelpapi: carrentals
Processing row 139.......... 	 31% complete
yelpapi: carrentals
Processing row 140.......... 	 32% complete
yelpapi: carrentals
Processing row 141.......... 	 32% complete
yelpapi: carrentals
Processing row 142.......... 	 32% complete
yelpapi: carrentals
Processing row 143.......... 	 32% complete
yelpapi: carrentals
Processing row 144.......... 	 33% complete
yelpapi: carrentals
Processing row 145.......... 	 33% complete
yelpapi: carrentals
Processing row 146.......... 	 33% compl

Processing row 260.......... 	 59% complete
yelpapi: carrentals
Processing row 261.......... 	 59% complete
yelpapi: carrentals
Processing row 262.......... 	 59% complete
yelpapi: carrentals
Processing row 263.......... 	 59% complete
yelpapi: carrentals
Processing row 264.......... 	 60% complete
yelpapi: carrentals
Processing row 265.......... 	 60% complete
yelpapi: carrentals
Processing row 266.......... 	 60% complete
yelpapi: carrentals
Processing row 267.......... 	 60% complete
yelpapi: carrentals
Processing row 268.......... 	 60% complete
yelpapi: carrentals
Processing row 269.......... 	 61% complete
yelpapi: carrentals
Processing row 270.......... 	 61% complete
yelpapi: carrentals
Processing row 271.......... 	 61% complete
yelpapi: carrentals
Processing row 272.......... 	 61% complete
yelpapi: carrentals
Processing row 273.......... 	 62% complete
yelpapi: carrentals
Processing row 274.......... 	 62% complete
yelpapi: carrentals
Processing row 275.......... 	 62% compl

Processing row 389.......... 	 88% complete
yelpapi: carrentals
Processing row 390.......... 	 88% complete
yelpapi: carrentals
Processing row 391.......... 	 88% complete
yelpapi: carrentals
Processing row 392.......... 	 88% complete
yelpapi: carrentals
Processing row 393.......... 	 89% complete
yelpapi: carrentals
Processing row 394.......... 	 89% complete
yelpapi: carrentals
Processing row 395.......... 	 89% complete
yelpapi: carrentals
Processing row 396.......... 	 89% complete
yelpapi: carrentals
Processing row 397.......... 	 90% complete
yelpapi: carrentals
Processing row 398.......... 	 90% complete
yelpapi: carrentals
Processing row 399.......... 	 90% complete
yelpapi: carrentals
Processing row 400.......... 	 90% complete
yelpapi: carrentals
Processing row 401.......... 	 91% complete
yelpapi: carrentals
Processing row 402.......... 	 91% complete
yelpapi: carrentals
Processing row 403.......... 	 91% complete
yelpapi: carrentals
Processing row 404.......... 	 91% compl

,Name,Review Count,Ratings,Categories,Bike Station Id
0,National Car Rental,84,3.5,"[{'alias': 'carrental', 'title': 'Car Rental'}]",1
1,Rental Car Center Boston,1,3.0,"[{'alias': 'carrental', 'title': 'Car Rental'}]",1
2,Avis Car Rental,240,1.5,"[{'alias': 'carrental', 'title': 'Car Rental'}]",1
3,Thrifty Car Rental,201,1.5,"[{'alias': 'carrental', 'title': 'Car Rental'}]",1
4,Advantage Rental Cars,117,1.0,"[{'alias': 'carrental', 'title': 'Car Rental'}]",1
...,...,...,...,...,...
467,National Car Rental,14,4.0,"[{'alias': 'carrental', 'title': 'Car Rental'}]",442
468,Thrifty Car Rental,2,2.5,"[{'alias': 'carrental', 'title': 'Car Rental'}]",442
469,Dollar Car Rental,21,2.5,"[{'alias': 'carrental', 'title': 'Car Rental'}]",442
470,Budget Car Rental,1,1.0,"[{'alias': 'carrental', 'title': 'Car Rental'}]",442


### Analyzing and Cleaning the data

#### 1. Condensing the categories columns

In [48]:
def condense_category_column (df):
    cats = []
    for row in list(df.index):
        cat = [category['alias'] for category in df['Categories'][row]]
        cats.append(cat)
    # cats
    df['Category'] = cats
    df.drop(['Categories'], axis=1, inplace=True)

In [51]:
condense_category_column(yelp_rentals)
yelp_rentals

,Name,Review Count,Ratings,Bike Station Id,Category
0,National Car Rental,84,3.5,1,[carrental]
1,Rental Car Center Boston,1,3.0,1,[carrental]
2,Avis Car Rental,240,1.5,1,[carrental]
3,Thrifty Car Rental,201,1.5,1,[carrental]
4,Advantage Rental Cars,117,1.0,1,[carrental]
...,...,...,...,...,...
467,National Car Rental,14,4.0,442,[carrental]
468,Thrifty Car Rental,2,2.5,442,[carrental]
469,Dollar Car Rental,21,2.5,442,[carrental]
470,Budget Car Rental,1,1.0,442,[carrental]


#### Inspecting the categories 

In [ ]:
# Parsing through the "Categories" column 'manually' as .unique() does not work with lists
cat_unravels = []
for category in yelp_rentals['Category']:
    try:
        cat_unravels.index(category);
    except: 
        cat_unravels.append(category);

In [ ]:
cat_unravels

#### Checking for null data and duplicates

In [53]:
# checking for null data - No null data
yelp_rentals.info() 
yelp_rentals.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             472 non-null    object 
 1   Review Count     472 non-null    int64  
 2   Ratings          472 non-null    float64
 3   Bike Station Id  472 non-null    int64  
 4   Category         472 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 18.6+ KB


,Review Count,Ratings,Bike Station Id
count,472.000000,472.000000,472.000000
mean,27.529661,2.359110,206.548729
std,46.246446,1.107666,138.603981
min,1.000000,1.000000,1.000000
25%,2.000000,1.500000,70.000000
50%,14.000000,2.000000,200.000000
75%,28.000000,3.000000,312.000000
max,240.000000,5.000000,442.000000


In [54]:
# checking for duplicates
len(yelp_rentals) - len(yelp_rentals.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=False))

# found 75 duplicates
# dealing with duplicates
yelp_rentals = yelp_rentals.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=True)

75

In [55]:
yelp_rentals

,Name,Review Count,Ratings,Bike Station Id,Category
0,National Car Rental,84,3.5,1,[carrental]
1,Rental Car Center Boston,1,3.0,1,[carrental]
2,Avis Car Rental,240,1.5,1,[carrental]
3,Thrifty Car Rental,201,1.5,1,[carrental]
4,Advantage Rental Cars,117,1.0,1,[carrental]
...,...,...,...,...,...
392,Budget Car Rental,1,1.0,441,[carrental]
393,National Car Rental,14,4.0,442,[carrental]
394,Thrifty Car Rental,2,2.5,442,[carrental]
395,Dollar Car Rental,21,2.5,442,[carrental]


In [57]:
# saving the file
yelp_rentals.to_csv('../data/yelp_car_rentals_df.csv', index=False) 

pd.read_csv('../data/yelp_car_rentals_df.csv')

,Name,Review Count,Ratings,Bike Station Id,Category
0,National Car Rental,84,3.5,1,['carrental']
1,Rental Car Center Boston,1,3.0,1,['carrental']
2,Avis Car Rental,240,1.5,1,['carrental']
3,Thrifty Car Rental,201,1.5,1,['carrental']
4,Advantage Rental Cars,117,1.0,1,['carrental']
...,...,...,...,...,...
392,Budget Car Rental,1,1.0,441,['carrental']
393,National Car Rental,14,4.0,442,['carrental']
394,Thrifty Car Rental,2,2.5,442,['carrental']
395,Dollar Car Rental,21,2.5,442,['carrental']


### Restaurants

In [48]:
yelp_restaurant_df = loop_yelp_api_by_search('restaurant')
yelp_restaurant_df

Processing row 0.......... 	 0% complete
yelpapi: restaurant
Processing row 1.......... 	 0% complete
yelpapi: restaurant
Processing row 2.......... 	 0% complete
yelpapi: restaurant
Processing row 3.......... 	 1% complete
yelpapi: restaurant
Processing row 4.......... 	 1% complete
yelpapi: restaurant
Processing row 5.......... 	 1% complete
yelpapi: restaurant
Processing row 6.......... 	 1% complete
yelpapi: restaurant
Processing row 7.......... 	 2% complete
yelpapi: restaurant
Processing row 8.......... 	 2% complete
yelpapi: restaurant
Processing row 9.......... 	 2% complete
yelpapi: restaurant
Processing row 10.......... 	 2% complete
yelpapi: restaurant
Processing row 11.......... 	 2% complete
yelpapi: restaurant
Processing row 12.......... 	 3% complete
yelpapi: restaurant
Processing row 13.......... 	 3% complete
yelpapi: restaurant
Processing row 14.......... 	 3% complete
yelpapi: restaurant
Processing row 15.......... 	 3% complete
yelpapi: restaurant
Processing row 16.

Processing row 131.......... 	 30% complete
yelpapi: restaurant
Processing row 132.......... 	 30% complete
yelpapi: restaurant
Processing row 133.......... 	 30% complete
yelpapi: restaurant
Processing row 134.......... 	 30% complete
yelpapi: restaurant
Processing row 135.......... 	 30% complete
yelpapi: restaurant
Processing row 136.......... 	 31% complete
yelpapi: restaurant
Processing row 137.......... 	 31% complete
yelpapi: restaurant
Processing row 138.......... 	 31% complete
yelpapi: restaurant
Processing row 139.......... 	 31% complete
yelpapi: restaurant
Processing row 140.......... 	 32% complete
yelpapi: restaurant
Processing row 141.......... 	 32% complete
yelpapi: restaurant
Processing row 142.......... 	 32% complete
yelpapi: restaurant
Processing row 143.......... 	 32% complete
yelpapi: restaurant
Processing row 144.......... 	 33% complete
yelpapi: restaurant
Processing row 145.......... 	 33% complete
yelpapi: restaurant
Processing row 146.......... 	 33% compl

Processing row 260.......... 	 59% complete
yelpapi: restaurant
Processing row 261.......... 	 59% complete
yelpapi: restaurant
Processing row 262.......... 	 59% complete
yelpapi: restaurant
Processing row 263.......... 	 59% complete
yelpapi: restaurant
Processing row 264.......... 	 60% complete
yelpapi: restaurant
Processing row 265.......... 	 60% complete
yelpapi: restaurant
Processing row 266.......... 	 60% complete
yelpapi: restaurant
Processing row 267.......... 	 60% complete
yelpapi: restaurant
Processing row 268.......... 	 60% complete
yelpapi: restaurant
Processing row 269.......... 	 61% complete
yelpapi: restaurant
Processing row 270.......... 	 61% complete
yelpapi: restaurant
Processing row 271.......... 	 61% complete
yelpapi: restaurant
Processing row 272.......... 	 61% complete
yelpapi: restaurant
Processing row 273.......... 	 62% complete
yelpapi: restaurant
Processing row 274.......... 	 62% complete
yelpapi: restaurant
Processing row 275.......... 	 62% compl

Processing row 389.......... 	 88% complete
yelpapi: restaurant
Processing row 390.......... 	 88% complete
yelpapi: restaurant
Processing row 391.......... 	 88% complete
yelpapi: restaurant
Processing row 392.......... 	 88% complete
yelpapi: restaurant
Processing row 393.......... 	 89% complete
yelpapi: restaurant
Processing row 394.......... 	 89% complete
yelpapi: restaurant
Processing row 395.......... 	 89% complete
yelpapi: restaurant
Processing row 396.......... 	 89% complete
yelpapi: restaurant
Processing row 397.......... 	 90% complete
yelpapi: restaurant
Processing row 398.......... 	 90% complete
yelpapi: restaurant
Processing row 399.......... 	 90% complete
yelpapi: restaurant
Processing row 400.......... 	 90% complete
yelpapi: restaurant
Processing row 401.......... 	 91% complete
yelpapi: restaurant
Processing row 402.......... 	 91% complete
yelpapi: restaurant
Processing row 403.......... 	 91% complete
yelpapi: restaurant
Processing row 404.......... 	 91% compl

,Name,Review Count,Ratings,Categories,Bike Station Id
0,Luna's Restaurant Italian and Latin Cuisine,122,4.5,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",0
1,Angela's Cafe - Orient Heights,136,4.5,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",0
2,New Saigon,201,4.5,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",0
3,Melodias,12,4.0,"[{'alias': 'colombian', 'title': 'Colombian'}]",0
4,Renegade's Pub,126,4.0,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",0
...,...,...,...,...,...
8752,Gigi,30,4.5,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",442
8753,Sorellina,747,4.0,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",442
8754,PURO ceviche bar,294,4.5,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",442
8755,GRECO,571,4.5,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",442


### Analyzing and Cleaning the data

#### 1. Condensing the categories columns

In [49]:
condense_category_column (yelp_restaurant_df)

#### Checking for null data and duplicates

In [50]:
# checking for null data - No null data
yelp_restaurant_df.info() 
yelp_restaurant_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8757 entries, 0 to 8756
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8757 non-null   object 
 1   Review Count     8757 non-null   int64  
 2   Ratings          8757 non-null   float64
 3   Bike Station Id  8757 non-null   int64  
 4   Category         8757 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 342.2+ KB


,Review Count,Ratings,Bike Station Id
count,8757.000000,8757.000000,8757.000000
mean,345.333676,3.989323,220.162727
std,556.441989,0.664238,127.796862
min,0.000000,0.000000,0.000000
25%,45.000000,4.000000,110.000000
50%,151.000000,4.000000,220.000000
75%,422.000000,4.500000,331.000000
max,5981.000000,5.000000,442.000000


In [ ]:
# checking for duplicates
len(yelp_restaurant_df) - len(yelp_restaurant_df.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=False))
# 7

yelp_restaurant_df = yelp_restaurant_df.drop_duplicates(subset=['Name', 'Bike Station Id'], ignore_index=True)

In [55]:
yelp_restaurant_df

,Name,Review Count,Ratings,Bike Station Id,Category
0,Luna's Restaurant Italian and Latin Cuisine,122,4.5,0,"[italian, latin, bars]"
1,Angela's Cafe - Orient Heights,136,4.5,0,"[mexican, breakfast_brunch]"
2,New Saigon,201,4.5,0,[vietnamese]
3,Melodias,12,4.0,0,[colombian]
4,Renegade's Pub,126,4.0,0,"[pubs, burgers]"
...,...,...,...,...,...
8745,Gigi,30,4.5,442,"[italian, cocktailbars]"
8746,Sorellina,747,4.0,442,"[italian, bars, mediterranean]"
8747,PURO ceviche bar,294,4.5,442,"[wine_bars, cocktailbars, seafood]"
8748,GRECO,571,4.5,442,"[greek, salad, soup]"


#### Inspecting the categories 

In [56]:
# Parsing through the "Categories" column 'manually' as .unique() does not work with lists
cat_unravels = []
for category in yelp_restaurant_df['Category']:
    try:
        cat_unravels.index(category);
    except: 
        cat_unravels.append(category);

18

15

7

3

20

...

352

1114

19

1118

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
cat_unravels

[['italian', 'latin', 'bars'],
 ['mexican', 'breakfast_brunch'],
 ['vietnamese'],
 ['colombian'],
 ['pubs', 'burgers'],
 ['tradamerican', 'breakfast_brunch'],
 ['seafood', 'tradamerican'],
 ['chinese'],
 ['sushi'],
 ['delis', 'pizza', 'coffee'],
 ['latin', 'bars'],
 ['hotels', 'tradamerican'],
 ['mexican', 'seafood', 'breakfast_brunch'],
 ['colombian', 'bars', 'buffets'],
 ['bars', 'colombian'],
 ['pizza'],
 ['italian', 'tradamerican', 'delis'],
 ['burgers', 'hotdogs'],
 ['mexican'],
 ['sandwiches'],
 ['speakeasies', 'raw_food', 'cocktailbars'],
 ['beer_and_wine', 'burgers', 'cocktailbars'],
 ['newamerican', 'seafood', 'breakfast_brunch'],
 ['tradamerican', 'gastropubs', 'beerbar'],
 ['sushi', 'asianfusion', 'bbq'],
 ['mexican', 'bars'],
 ['italian'],
 ['cafes'],
 ['cheese', 'delis'],
 ['coffee', 'breakfast_brunch', 'juicebars'],
 ['sandwiches', 'coffee', 'juicebars'],
 ['bakeries', 'salad', 'sandwiches'],
 ['italian', 'lounges'],
 ['bars', 'tradamerican', 'seafood'],
 ['mexican', 'sal

In [63]:
# saving the file
yelp_restaurant_df.to_csv('../data/yelp_restaurant_df.csv', index=False) 

pd.read_csv('../data/yelp_restaurant_df.csv')

,Name,Review Count,Ratings,Bike Station Id,Category
0,Luna's Restaurant Italian and Latin Cuisine,122,4.5,0,"['italian', 'latin', 'bars']"
1,Angela's Cafe - Orient Heights,136,4.5,0,"['mexican', 'breakfast_brunch']"
2,New Saigon,201,4.5,0,['vietnamese']
3,Melodias,12,4.0,0,['colombian']
4,Renegade's Pub,126,4.0,0,"['pubs', 'burgers']"
...,...,...,...,...,...
8745,Gigi,30,4.5,442,"['italian', 'cocktailbars']"
8746,Sorellina,747,4.0,442,"['italian', 'bars', 'mediterranean']"
8747,PURO ceviche bar,294,4.5,442,"['wine_bars', 'cocktailbars', 'seafood']"
8748,GRECO,571,4.5,442,"['greek', 'salad', 'soup']"


# Comparing Results

## Which API provided you with more complete data? Provide an explanation. 


- Yelp provided information about the reviews and rating of each of the businesses. FourSquares provided information about the opening hours of the businesses (not collected, but observed in the .json() file) and the associated brands of the business. I would rank them about the same in terms of Quality of information. 

- FourSquares provided more data in general but a lot of that was non-unique records which required a lot of data cleaning. I go into details of that in 'Data Joining.'


- In summary: Yelp provided more quality data.


## Get the top 10 restaurants according to their rating

Only Yelp API included ratings and reviewed for businesses

* Note that if we just sort by ratings, we include restaurants that do not have a large review count.

In [60]:
yelp_restaurant_df.sort_values('Ratings', ascending=False)[:10][['Name', 'Ratings', 'Review Count']]
yelp_restaurant_df['Review Count'].describe()

,Name,Ratings,Review Count
3368,Thazia,5.0,4
330,Mei Mei Dumplings,5.0,11
2193,Don Tequeno & Dona Arepa,5.0,8
7093,FoodDy's Kitchen,5.0,16
7509,Madhouse Cafe,5.0,6
8205,Big Greg's House,5.0,1
8204,Avenue Deli,5.0,4
1372,Mom's Tiffin,5.0,1
5061,Sp Cafe,5.0,2
4540,Tukure Nepali Cuisine,5.0,6


count    8750.000000
mean      345.515314
std       556.600432
min         0.000000
25%        45.000000
50%       151.000000
75%       422.000000
max      5981.000000
Name: Review Count, dtype: float64

* Sort only the restaurants that have an above average review count.

In [61]:
# Filter by review_count greater than the mean
filter_by_review_count = yelp_restaurant_df['Review Count'] > yelp_restaurant_df['Review Count'].mean()

# Use group by to group to avoid restaurants that show up for more than 1 bike location
yelp_restaurant_df[filter_by_review_count].\
groupby('Name').agg({'Ratings': 'mean'}).sort_values('Ratings', ascending=False)[:10]

,Ratings
Name,
Ciao! Pizza & Pasta,5.0
3 Little Figs,4.5
Juliet,4.5
Tasty Mo:Mo,4.5
Ostra,4.5
Chilacates Mexican Street Food,4.5
Tenoch Mexican,4.5
Cha Yen Thai Cookery,4.5
Oleana Restaurant,4.5
